<a href="https://colab.research.google.com/github/Danis2019/TestTaskDLS/blob/main/TestTaskDLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Тестовое задание: симулировать динамическое начисление скидки покупателям магазина согласно условиям:
** 1.диапазон скидки от 0% до b_max%; размер выбирается случным образом, но при условии, что
** 2.общая средневзвешенная скидка по текущей и всем предидущим покупкам на 25% меньше чем b_max/3
* опционно: генерировать случайный размер входящих покупок не по равномерному распределению, а по распределению Пуассона с параметром mu=1.8

In [ ]:
import pandas as pd
from scipy.stats import poisson
from random import randint
import numpy as np

In [ ]:
def puasson():
  return poisson.rvs(mu=1.8, size=10)

In [ ]:
b_max = randint(10, 50)
print(f"b_max = {b_max}")

b_max = 23


In [ ]:
print('список цен/покупок:')
prices_array = puasson() * 10 + 10
prices_array

список цен/покупок:


array([40, 20, 20, 30, 20, 30, 10, 70, 30, 40])

In [ ]:
# # Список покупок и их персональных скидок
# df = pd.DataFrame({ 'Цена': puasson() * 10 + 10,
#                     'Скидка': puasson()})
# # Необходимо обработать значение скидки = 0, чтобы не было деления на 0
# df['Вес скидки'] = np.where(df['Скидка'] !=0, df['Цена'] * df['Скидка'] / 100, 0)
# df

In [ ]:
# Вес это цена помноженная на скидку деленную на 100
def create_disсount_v1(df, disounts, weights, product_price):
  if len(df) != 0:
    d = df[disounts]
    w = df[weights]
    # Если принять максимальную скидку за X, то можно вывести квадратное уравнение с коэффициентами, представленными на следующей строчке
    coeff = (product_price/100, product_price/100 * b_max / 4, (d * w).sum() - b_max / 4 * w.sum())
    # Выведение этого квадратного уравнения можно найти на Github, либо я вышлю Дине
    person_disount = np.roots(coeff)
    return randint(0, int(max(person_disount))), max(person_disount)
  else:
    person_disount =randint(0, int(b_max / 4))
    return person_disount, b_max / 4

In [ ]:
# Вес это цена
def create_disсount_v2(df, disounts, weights, product_price):
  if len(df) != 0:
    d = df[disounts]
    w = df[weights]
    # Если принять максимальную скидку за X, то можно вывести формулу для максимального значения скидки
    person_disount =  (b_max / 4 * (w.sum() + product_price) - (d * w).sum()) / price
    return randint(0, int(person_disount)), person_disount
  else:
    person_disount =randint(0, int(b_max / 4))
    return person_disount, b_max / 4

* Для случаев, где вес это цена на скидку

In [ ]:
# Тут начинает возникать проблема, если первая скидка равна 0, то и остальные скидки становяться равными 0
df = pd.DataFrame(columns=['Цена', 'Скидка', 'Вес скидки', 'Максимально доступная скидка'])
for price in prices_array:
  discount, max_discount = create_disсount_v1(df, 'Скидка', 'Вес скидки', price)
  weight = price * discount / 100
  df = pd.concat([pd.DataFrame([[price, discount, weight, max_discount]], columns=df.columns), df], ignore_index=True)
df

,Цена,Скидка,Вес скидки,Максимально доступная скидка
0,40,2,0.8,3.255100
1,30,3,0.9,3.377916
2,70,1,0.7,1.215221
3,10,2,0.2,4.871975
4,30,1,0.3,1.681932
5,20,0,0.0,2.322656
6,30,0,0.0,1.681932
7,20,1,0.2,1.843647
8,20,0,0.0,1.843647
9,40,4,1.6,5.750000


* Для случаев, где вес это цена

In [ ]:
df = pd.DataFrame(columns=['Цена', 'Скидка', 'Вес скидки', 'Максимально доступная скидка'])
for price in prices_array:
  discount, max_discount = create_disсount_v2(df, 'Скидка', 'Цена', price)
  weight = price
  df = pd.concat([pd.DataFrame([[price, discount, weight, max_discount]], columns=df.columns), df], ignore_index=True)
df

,Цена,Скидка,Вес скидки,Максимально доступная скидка
0,30,4,30,20.083333
1,50,15,50,21.800000
2,20,25,20,57.625000
3,30,9,30,41.583333
4,30,6,30,38.833333
5,50,2,50,20.050000
6,30,3,30,21.833333
7,40,8,40,17.812500
8,20,5,20,23.125000
9,50,3,50,8.750000


* Как функция

In [ ]:
def add_discount_to_df(df, price):
  discount, max_discount = create_disсount_v1(df, 'Скидка', 'Вес скидки', price)
  weight = price * discount / 100
  df = pd.concat([pd.DataFrame([[price, discount, weight, max_discount]], columns=df.columns), df], ignore_index=True)